<a href="https://colab.research.google.com/github/AkiseF/Bioinformatica/blob/main/Practica12_Modelo_QSAR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Elaboración: Parra Mancilla José Ramón \
Curso: Bioinformática \
Profesor: Jorge Luis Rosas Trigueros \
Semestre: 2025/2 \
ESCOM IPN \
https://colab.research.google.com/drive/1l5F-wRmjNgOV-T62YUmbLEZ3C0W8QLOX?usp=sharing


---

In [2]:
# ================================
# 1. Instalación de librerías
# ================================
!pip install -q rdkit pandas pysr chembl_webresource_client

# ================================
# 2. Importar librerías
# ================================
import pandas as pd
from chembl_webresource_client.new_client import new_client
from rdkit import Chem
from rdkit.Chem import Descriptors
from pysr import PySRRegressor
from google.colab import files
import numpy as np

# ================================
# 3. Cargar archivo FDA con moléculas aprobadas
# ================================
print("Sube el archivo 'Launched 30may25.txt' con moléculas FDA")
uploaded = files.upload()
fda_filename = list(uploaded.keys())[0]

df_fda = pd.read_csv(fda_filename, sep="\t")
print(f"Archivo FDA cargado: {fda_filename}")
print(f"Columnas disponibles: {df_fda.columns.tolist()}")
print(f"Total moléculas FDA: {len(df_fda)}")

[juliapkg] Found dependencies: /usr/local/lib/python3.11/dist-packages/pysr/juliapkg.json
[juliapkg] Found dependencies: /usr/local/lib/python3.11/dist-packages/juliacall/juliapkg.json
[juliapkg] Found dependencies: /usr/local/lib/python3.11/dist-packages/juliapkg/juliapkg.json
[juliapkg] Locating Julia =1.10.0, ^1.10.3
[juliapkg] Using Julia 1.10.9 at /usr/local/bin/julia
[juliapkg] Using Julia project at /root/.julia/environments/pyjuliapkg
[juliapkg] Writing Project.toml:
             [deps]
             SymbolicRegression = "8254be44-1295-4e6a-a16d-46603ac705cb"
             Serialization = "9e88b42a-f829-5b0c-bbe9-9e923198166b"
             PythonCall = "6099a3de-0909-46bc-b1f4-468b9a2dfc0d"
             OpenSSL_jll = "458c3c95-2e84-50aa-8efc-19380b2a3a95"
             [compat]
             SymbolicRegression = "~1.11"
             Serialization = "^1"
             PythonCall = "=0.9.25"
             OpenSSL_jll = "~3.0"
[juliapkg] Installing packages:
             import Pkg
    

Saving Launched 30may25.txt to Launched 30may25.txt
Archivo FDA cargado: Launched 30may25.txt
Columnas disponibles: ['Name', 'MOA', 'Target', 'SMILES', 'Phase']
Total moléculas FDA: 2427


In [3]:
# ================================
# 4. Descargar datos bioactivos IC50 desde ChEMBL para NUPR1, HSD11B1 y FFAR1
# ================================
def fetch_target_data(uniprot):
    targets = new_client.target.filter(target_components__accession=uniprot, target_organism="Homo sapiens")
    if not targets:
        print(f"⚠️ No se encontró target para {uniprot}")
        return pd.DataFrame()
    chembl_id = targets[0]['target_chembl_id']
    activities = new_client.activity.filter(
        target_chembl_id=chembl_id,
        type="IC50",
        standard_relation="=",
        assay_type="B"
    ).only(['molecule_chembl_id', 'canonical_smiles', 'standard_value', 'standard_units'])
    df = pd.DataFrame.from_records(activities)
    df = df[df['standard_value'].notna()]
    df = df[df['standard_units'].str.lower() == 'nm']
    df['standard_value'] = df['standard_value'].astype(float)
    df['pIC50'] = -np.log10(df['standard_value'] * 1e-9)
    df['target'] = uniprot
    return df[['molecule_chembl_id', 'canonical_smiles', 'pIC50', 'target']]

proteins = ["O60356", "P28845", "Q0IJ71"]
dfs = []
for prot in proteins:
    print(f"Descargando datos para {prot}...")
    dfs.append(fetch_target_data(prot))
df_act = pd.concat(dfs, ignore_index=True)
print(f"Total compuestos bioactivos obtenidos: {len(df_act)}")

Descargando datos para O60356...
⚠️ No se encontró target para O60356
Descargando datos para P28845...
Descargando datos para Q0IJ71...
⚠️ No se encontró target para Q0IJ71
Total compuestos bioactivos obtenidos: 4261


In [4]:
# ================================
# 5. Calcular descriptores moleculares con RDKit para los datos bioactivos
# ================================
df_act['mol'] = df_act['canonical_smiles'].apply(Chem.MolFromSmiles)
descriptor_names = ["MolWt", "MolLogP", "NumHDonors", "NumHAcceptors", "TPSA"]
for desc in descriptor_names:
    df_act[desc] = df_act['mol'].apply(getattr(Descriptors, desc))

In [5]:
# ================================
# 6. Entrenar modelos QSAR simbólicos con PySR para cada proteína
# ================================
models = {}
for target in df_act['target'].unique():
    print(f"\nEntrenando modelo QSAR para proteína: {target}")
    sub = df_act[df_act['target'] == target]
    X = sub[descriptor_names]
    y = sub['pIC50']
    model = PySRRegressor(
        model_selection="best",
        niterations=40,
        binary_operators=["+", "-", "*", "/"],
        unary_operators=["sqrt", "log", "exp"],
        loss="loss(x,y) = (x-y)^2",
        maxsize=20,
        verbosity=1,
    )
    model.fit(X, y)
    print(f"Mejor expresión para {target}:")
    print(model.get_best())
    models[target] = model

/usr/local/lib/python3.11/dist-packages/pysr/sr.py:1036: FutureWarning: `loss` has been renamed to `elementwise_loss` in PySRRegressor. Please use that instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pysr/sr.py:2811: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(
Compiling Julia backend...
Compiling Julia backend...
INFO:pysr.sr:Compiling Julia backend...



Entrenando modelo QSAR para proteína: P28845


[ Info: Started!



Expressions evaluated per second: 7.680e+03
Progress: 36 / 1240 total iterations (2.903%)
════════════════════════════════════════════════════════════════════════════════════════════════════
───────────────────────────────────────────────────────────────────────────────────────────────────
Complexity  Loss       Score      Equation
1           1.287e+00  0.000e+00  y = 7.3004
4           1.183e+00  2.810e-02  y = log(MolWt) + 1.3119
7           1.176e+00  1.989e-03  y = log((MolWt / 0.54218) * sqrt(MolLogP))
9           1.167e+00  3.898e-03  y = sqrt(sqrt(sqrt(sqrt(exp(MolLogP))))) + log(MolWt)
11          1.150e+00  7.203e-03  y = ((sqrt(sqrt(MolWt - 2.8957)) / 0.64281) - 0.19549) / 0...
                                      .92245
13          1.149e+00  3.109e-04  y = (((log(sqrt(MolWt - 2.0101)) / -1.0776) - -0.78038) + ...
                                      0.31228) / -0.22965
14          1.149e+00  2.673e-04  y = ((((sqrt(NumHDonors) * 0.028371) + log(MolWt)) * 0.885...
      

[ Info: Final population:
[ Info: Results saved to:


───────────────────────────────────────────────────────────────────────────────────────────────────
Complexity  Loss       Score      Equation
1           1.287e+00  0.000e+00  y = 7.3004
4           1.169e+00  3.194e-02  y = log(MolWt) * 1.2198
5           1.150e+00  1.634e-02  y = sqrt(sqrt(MolWt) * 2.6641)
6           1.149e+00  1.386e-03  y = log(MolWt + -175.75) + 1.9212
7           1.149e+00  2.013e-04  y = sqrt(sqrt(MolWt + -190.98)) + 3.5203
8           1.147e+00  1.612e-03  y = sqrt(sqrt(MolLogP) + sqrt(MolWt)) * 1.5579
9           1.143e+00  3.302e-03  y = sqrt(MolLogP + ((sqrt(MolWt) / 0.38216) + -2.9993))
12          1.140e+00  1.023e-03  y = sqrt(sqrt(MolWt) + ((NumHAcceptors + (sqrt(MolWt) + Mo...
                                      lLogP)) * 1.1978))
14          1.136e+00  1.498e-03  y = sqrt(sqrt(MolWt) + ((NumHAcceptors + (NumHDonors + sqr...
                                      t(MolWt))) + (MolLogP * 2.0946)))
15          1.134e+00  1.687e-03  y = sqrt(((MolLogP +

In [8]:
# ================================
# 7. Calcular descriptores para moléculas FDA y predecir actividades con los modelos QSAR
# ================================

# ================================
# LIMPIEZA DE SMILES PARA MOLÉCULAS FDA
# ================================

# Asegurarse que todos los SMILES son strings y eliminar espacios o comas al final
df_fda['SMILES'] = df_fda['SMILES'].astype(str).str.strip().str.rstrip(',')

# Filtrar SMILES que no se pueden convertir a moléculas
def safe_mol(smiles):
    try:
        mol = Chem.MolFromSmiles(smiles)
        return mol
    except:
        return None

df_fda['mol'] = df_fda['SMILES'].apply(safe_mol)
df_fda = df_fda[df_fda['mol'].notnull()].reset_index(drop=True)

print(f"✅ Moléculas válidas después del filtrado: {len(df_fda)}")

print("\nCalculando descriptores para moléculas FDA...")
df_fda['mol'] = df_fda['SMILES'].apply(Chem.MolFromSmiles)

for desc in descriptor_names:
    df_fda[desc] = df_fda['mol'].apply(getattr(Descriptors, desc))

print("\nRealizando predicciones para cada proteína...")
for target, model in models.items():
    print(f"Prediciendo actividad para {target}...")
    X_fda = df_fda[descriptor_names].fillna(0)  # llenar NaN si hay
    df_fda[f"pIC50_pred_{target}"] = model.predict(X_fda)

✅ Moléculas válidas después del filtrado: 1051

Calculando descriptores para moléculas FDA...

Realizando predicciones para cada proteína...
Prediciendo actividad para P28845...


In [9]:
# ================================
# 8. Filtrar candidatos potenciales para reposicionamiento
# ================================
print("\nCandidatos potenciales con pIC50 predicha > 6 (actividad significativa):")
threshold = 6.0
for target in models.keys():
    candidates = df_fda[df_fda[f"pIC50_pred_{target}"] > threshold]
    print(f"\nProteína: {target} - {len(candidates)} candidatos")
    display(candidates[['Name', 'MOA', 'Target', 'SMILES', f'pIC50_pred_{target}']].head(10))


Candidatos potenciales con pIC50 predicha > 6 (actividad significativa):

Proteína: P28845 - 1024 candidatos


,Name,MOA,Target,SMILES,pIC50_pred_P28845
0,abacavir,nucleoside reverse transcriptase inhibitor,NaN,Nc1nc(NC2CC2)c2ncn([C@@H]3C[C@H](CO)C=C3)c2n1 ...,6.900715
1,abamectin,benzodiazepine receptor agonist,"GABBR1, GABBR2",[H][C@@]12OC\C3=C/C=C/[C@H](C)[C@H](O[C@@]4([H...,8.240885
2,abiraterone,androgen biosynthesis inhibitor,"CYP11B1, CYP17A1",C[C@]12CC[C@H]3[C@@H](CC=C4C[C@@H](O)CC[C@]34C...,7.143920
3,abiraterone-acetate,androgen biosynthesis inhibitor,CYP17A1,CC(=O)O[C@H]1CC[C@]2(C)[C@H]3CC[C@@]4(C)[C@@H]...,7.282456
4,acarbose,glucosidase inhibitor,"AMY2A, MGAM",C[C@H]1O[C@H](O[C@@H]2[C@@H](CO)O[C@H](O[C@@H]...,7.892445
5,aceclidine,acetylcholine receptor agonist,"CHRM1, CHRM2, CHRM3, CHRM4, CHRM5","CC(=O)O[C@H]1CN2CCC1CC2 |&1:4,r|",6.259149
6,acenocoumarol,vitamin K antagonist,VKORC1,CC(=O)C[C@@H](c1ccc(cc1)[N+]([O-])=O)c1c(O)c2c...,7.157152
8,aclarubicin,topoisomerase inhibitor,"TOP1, TOP2A",CC[C@@]1(O)C[C@H](O[C@H]2C[C@@H]([C@H](O[C@H]3...,8.171975
9,acrisorcin,other antifungal,NaN,CCCCCCc1ccc(O)cc1O.Nc1c2ccccc2nc2ccccc12,7.272936
10,adenosine-phosphate,adenosine receptor agonist,"ACSL1, ACSS1, ACSS2, ADCY1, ADK, CREB1, FBP1, ...",Nc1ncnc2n(cnc12)[C@@H]1O[C@H](COP(O)(O)=O)[C@@...,7.135888
